# 场景三：面向文档的对话机器人

In [16]:
!ollama pull llama2-chinese:13b

pulling manifest
pulling 8359bebea988... 100% |██████████████████| (7.4/7.4 GB, 63 TB/s)        
pulling 65c6ec5c6ff0... 100% |████████████████████| (45/45 B, 2.2 MB/s)        
pulling dd36891f03a0... 100% |████████████████████| (31/31 B, 1.2 MB/s)        
pulling f94f529485e6... 100% |███████████████████| (382/382 B, 22 MB/s)        
verifying sha256 digest
writing manifest
removing any unused layers
success


In [ ]:
%pip install langchain
%pip install faiss-cpu networkx

In [15]:
from operator import itemgetter

from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.chat_models import ChatOllama
from langchain.embeddings import OllamaEmbeddings
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.schema import format_document
from langchain.vectorstores import FAISS
from langchain.document_loaders import ArxivLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 加载 arXiv 上的论文《ReAct: Synergizing Reasoning and Acting in Language Models》
loader = ArxivLoader(query="2210.03629", load_max_docs=1)
docs = loader.load()

# 把文本分割成 200 字一组的切片
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)
chunks = text_splitter.split_documents(docs)

# 构建 FAISS 向量存储和对应的 retriever
vs = FAISS.from_documents(chunks[:10], OllamaEmbeddings(model="llama2-chinese:13b"))
# vs.similarity_search("What is ReAct")
retriever = vs.as_retriever()

# 构建 Document 转文本段落的工具函数
DEFAULT_DOCUMENT_PROMPT = PromptTemplate.from_template(template="{page_content}")
def _combine_documents(
    docs, document_prompt=DEFAULT_DOCUMENT_PROMPT, document_separator="\n\n"
):
    doc_strings = [format_document(doc, document_prompt) for doc in docs]
    return document_separator.join(doc_strings)

# 准备 Model I/O 三元组
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOllama(model="llama2-chinese:13b")

# 构建 RAG 链
chain = (
    {
        "context": retriever | _combine_documents,
        "question": RunnablePassthrough()
    }
    | prompt
    | model
    | StrOutputParser()
)
chain.invoke("什么是 ReAct？")

'ReAct：SYNERGIZING REASONING AND ACTING IN LANGUAGE MODELS，是一篇由Department of Computer Science, Princeton University和Google Research的Brain team合作在ICLR 2023发表的研究论文。ReAct旨在探索LLMs用于生成任务解释轨迹和任务特定动作的方法，以实现更加优化的解决问题之间的交织关系。该项目使用了简单的Wikipedia API来并行地生成人类可读的解释轨迹和任务轨迹，从而获得更好的探索性能和解决问题效果。'

In [4]:
from langchain.llms import Ollama
from langchain.memory import ConversationKGMemory

llm = Ollama(model="llama2-chinese:13b")
memory = ConversationKGMemory(llm=llm)
memory.save_context({"input": "LangChain 是什么"}, {"output": "LangChain 是一个大语言模型的应用开发框架，目前有 Python 和 JavaScript SDK"})
memory.save_context({"input": "Ollama 又是什么"}, {"output": "Ollama 是一个跨平台的运行大语言模型的工具软件，目前可以在 Linux 和 macOS 平台上运行"})

memory.load_memory_variables({"input": "LangChain 是啥？"})

{'history': 'On LangChain: LangChain application development framework for large language models.'}